In [3]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from random import randrange
from scipy.optimize import *
import matplotlib.pyplot as plt

Cuda is not available!
logicqubit version 1.5.8


In [4]:
gates = Gates()

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [12]:
IIII = ID.kron(ID).kron(ID).kron(ID)
XXXX = X.kron(X).kron(X).kron(X)
YYYY = Y.kron(Y).kron(Y).kron(Y)
ZZZZ = Z.kron(Z).kron(Z).kron(Z)
signals = [ZZZZ.get()[i,i] for i in range(len(ZZZZ.get()))]
signals # used in <psi|ZZZZ|psi>

[1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1]

In [6]:
H = IIII*3 + ZZZZ*8

min(scipy.linalg.eig(H.get())[0])

(-5+0j)

In [7]:
def _ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        reg[1].CNOT(reg[0])
        for j in range(n_qubits):
            reg[i].RY(params[j])
            
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_4q(q1, q2, q3, q4, params):
    q1.RY(params[0])
    q2.RY(params[1])
    q3.RY(params[2])
    q4.RY(params[3])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q4.CNOT(q3)
    q1.RX(params[4])
    q2.RX(params[5])
    q3.RX(params[6])
    q4.RX(params[7])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q4.CNOT(q3)
    q1.RY(params[8])
    q2.RY(params[9])
    q3.RY(params[10])
    q4.RY(params[11])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q4.CNOT(q3)
    q1.RY(params[12])
    q2.RY(params[13])
    q3.RY(params[14])
    q4.RY(params[15])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q4.CNOT(q3)

In [30]:
def expectation_4q(params):
    logicQuBit  = LogicQuBit(4)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    q4 = Qubit()

    ansatz_4q(q1,q2,q2,q4,params)
    #ansatz([q1,q2,q3,q4],params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

initial_values = np.random.uniform(-np.pi, np.pi, 16)
minimum = minimize(expectation_4q, initial_values, method='Nelder-Mead',options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 0.97108118, -0.26330511, -1.30749121, -0.6436676 , -3.14159267,
         1.23238916,  0.63362969, -1.27557381,  0.24058204,  0.61156406,
        -0.61156407,  2.49663256,  0.96096669, -0.58769253,  1.62413606,
        -1.57079633],
       [ 0.97108118, -0.26330511, -1.30749121, -0.6436676 , -3.14159267,
         1.23238916,  0.63362969, -1.27557381,  0.24058204,  0.61156406,
        -0.61156407,  2.49663256,  0.96096669, -0.58769253,  1.62413606,
        -1.57079633],
       [ 0.97108118, -0.26330511, -1.30749121, -0.6436676 , -3.14159267,
         1.23238916,  0.63362969, -1.27557381,  0.24058204,  0.61156406,
        -0.61156407,  2.49663256,  0.96096669, -0.58769253,  1.62413606,
        -1.57079633],
       [ 0.97108118, -0.26330511, -1.30749121, -0.6436676 , -3.14159267,
         1.23238916,  0.63362969, -1.27557381,  0.24058204,  0.61156406,
        -0.61156407,  2.49663256,  0.96096669, -0.58769253,  1.62413606,
        -1.57079633],
       [ 0.97108118

In [20]:
def expectation_value(measurements):
    probabilities = np.array(measurements)
    states = signals
    expectation = np.sum(states * probabilities) # <psi|ZZZZ|psi>
    return expectation

def sigma_xxxx(params):
    logicQuBit  = LogicQuBit(4, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    q4 = Qubit()
    
    ansatz_4q(q1,q2,q3,q4,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    q3.RY(-pi/2)
    q4.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3,q4])
    result = expectation_value(result)
    return result

def sigma_yyyy(params):
    logicQuBit  = LogicQuBit(4, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    q4 = Qubit()
    
    ansatz_4q(q1,q2,q3,q4,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    q3.RX(pi/2)
    q4.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3,q4])
    result = expectation_value(result)
    return result

def sigma_zzzz(params):
    logicQuBit  = LogicQuBit(4, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    q4 = Qubit()
    
    ansatz_4q(q1,q2,q3,q4,params)
          
    result = logicQuBit.Measure([q1,q2,q3,q4])
    result = expectation_value(result)
    return result

def expectation_energy(params):
    xxxx =  sigma_xxxx(params)
    yyyy =  sigma_yyyy(params)
    zzzz =  sigma_zzzz(params)

    result = 3 + 8*zzzz
    return result

In [23]:
initial_values = np.random.uniform(-np.pi, np.pi, 16)
minimum = minimize(expectation_energy, initial_values, method='Nelder-Mead')
print(minimum)

 final_simplex: (array([[-3.14158631e+00, -1.56722459e+00,  4.30413762e-01,
         9.60212067e-02,  3.14158852e+00,  1.56202882e+00,
        -2.35871198e+00,  3.21365907e+00,  1.15642881e-06,
         1.57489919e+00, -9.75093207e-01,  3.13672123e+00,
         2.87586517e+00, -2.69623135e-03, -5.14798803e-02,
        -1.57040064e+00],
       [-3.14159290e+00, -1.56719816e+00,  4.30399183e-01,
         9.60238298e-02,  3.14158941e+00,  1.56201745e+00,
        -2.35870998e+00,  3.21363982e+00,  2.77304237e-06,
         1.57491768e+00, -9.75103211e-01,  3.13669459e+00,
         2.87585784e+00, -2.69622332e-03, -5.14830310e-02,
        -1.57040753e+00],
       [-3.14159594e+00, -1.56716079e+00,  4.30383177e-01,
         9.60275892e-02,  3.14159095e+00,  1.56197556e+00,
        -2.35872335e+00,  3.21365304e+00, -2.24314318e-06,
         1.57496858e+00, -9.75105170e-01,  3.13663393e+00,
         2.87590226e+00, -2.69620394e-03, -5.14862760e-02,
        -1.57040722e+00],
       [-3.14158731e

In [24]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [25]:
params = np.random.uniform(-np.pi, np.pi, 16)
last_params = np.zeros(16)

In [27]:
lr = 0.05
err = 1
while err > 1e-15:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

<ipython-input-24-cef20e405c15>:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


0.08546257568133853
0.15416069170439184
0.0010030077885998523
0.005848087390222023
0.0014074649937678707
0.00037367540024457746
0.000525058931988287
0.0003059735653656581
0.00021555852567467504
0.00013823997861606152
9.094510252460886e-05
5.9029449401171163e-05
3.842826257582921e-05
2.4959655773759692e-05
1.6213877932491627e-05
1.0527286879780107e-05
6.83443283255869e-06
4.43633101898655e-06
2.8795013184979013e-06
1.868905151303224e-06
1.2129531050675268e-06
7.872115472817853e-07
5.108967460198954e-07
3.315667485015439e-07
2.151822029450301e-07
1.396497364414273e-07
9.063018180377469e-08
5.881727571654949e-08
3.81712628172437e-08
2.4772387252625094e-08
1.607677746484626e-08
1.0433500063911083e-08
6.771128102656121e-09
4.3943220173758846e-09
2.8518238881503066e-09
1.8507753107854796e-09
1.201114319115959e-09
7.794980216857539e-10
5.058791163747856e-10
3.2830427265650997e-10
2.1306334474502364e-10
1.3827272660194012e-10
8.973621845598245e-11
5.823808102434214e-11
3.779376811507973e-11
2.

In [28]:
expectation_energy(params)

(-4.999999999999998+0j)